In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from collections import deque
from typing import List, Tuple

rng = np.random.default_rng(432)


[Introduction to Multi-Armed Bandits by Aleksandrs Slivkins](https://arxiv.org/abs/1904.07272)
### Assumptions

* __IID rewards__: the reward for each arm is drawn independently from a fixed distribution that depends on the arm but not on the round t. The rewards do not vary over time.
* __bandit feedback__: The algorithm observes only the reward for the selected action, and nothing else. In particular, it does not observe rewards for other actions that could have been selected.

* independent priors and posteriors

* TODO: add rewards are bernoulli

## Thompson sampling algorithm (Beta-Bernoulli)

Assign the initial parameters $\alpha$ and $\beta$ to each arm. 


For each round $t$ = 1,2,... $T$, do:
1. Sample a mean reward for each arm $k$ using its parameters

    $\mu_t(a_k) \sim  Beta(\alpha_k, \beta_k)$ 

2. Select the arm with the largest mean reward $a_{k^*}$
3. Observe the reward $r_t$ of that arm and update its parameters

    $(\alpha_{k^*}, \beta_{k^*}) \leftarrow (\alpha_{k^*} + r_t, \beta_{k^*} + 1 - r_t)$

Assign a Beta prior distribution to each arm
Initialize each arm with a Beta distributon as prior for the mean reward

In [2]:
n_bandists = 5
# expected rewards sampled from a Beta(1, 1) dist

expected_rewards = rng.beta(1, 1, size=n_bandists)
bandits = [stats.bernoulli(p=p) for p in expected_rewards]
best_expected_reward = max(expected_rewards)
best_arm = np.argmax(expected_rewards)
assert expected_rewards[best_arm] == best_expected_reward 

In [3]:
# total rounds
T = 50

# num_bandits, (alpha, beta) parameters
initial_parameters = np.ones((n_bandists, 2))
parameters = [initial_parameters]
print(f' #bandits, #parameters per bandit: {initial_parameters.shape}')

 #bandits, #parameters per bandit: (5, 2)


In [4]:

# refactor loop
# plot  25 x 2, titple t round

In [5]:


H: List[Tuple[int, int]] = []
beliefs = []
for t in range(0, T):
    parameters_t = parameters[t].copy()
    samples_p = [stats.beta(p[0], p[1]).rvs() for p in parameters_t]
    k = np.argmax(samples_p)
    r = bandits[k].rvs()
    H.append((k, r))
    if r == 1:
        parameters_t[k, 0] += 1
    else:
        parameters_t[k, 1] += 1
    parameters.append(parameters_t)

Regret: $R(T) = \mu^* \times T - \sum_t^T\mu(a_t) $

In [6]:
def regret(expected_rewards, history) -> float:
    best_arm_benchmark = max(expected_rewards) * len(history)
    cumulative_reward = sum(reward for arm, reward in history)
    return best_arm_benchmark - cumulative_reward


regret(expected_rewards, H)

# TODO: add expected regret, it could be an histogram

1.5868815814590391

In [7]:
expected_rewards

array([0.13003388, 0.38059356, 0.21934638, 0.69844892, 0.81173763])

In [8]:
# TODO: check possible error in document

In [9]:
# expected_rewards = [0.10, 0.20, 0.80, 0.60]